In [25]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [26]:
app = dash.Dash()

## Load dfs and clean

In [27]:
feminicidios = pd.read_csv('../BD-sucias/feminicidios-corregida.csv', low_memory=False)
feminicidios.fecha_recepcion = pd.to_datetime(feminicidios.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
feminicidios['año_recepcion'] = feminicidios['fecha_recepcion'].dt.year
nas = feminicidios[(feminicidios.TipoRelacion.isna()==True)|(feminicidios.TipoRelacion=='Seleccione')].index
feminicidios.loc[nas,'TipoRelacion'] = 'Desconocido'
com = feminicidios[(feminicidios.TipoRelacion=='En la comunidad')].index
feminicidios.loc[com,'TipoRelacion'] = 'Comunidad'

In [28]:
servicios = pd.read_csv('../BD-sucias/servicio_10_febrero_2022.csv', low_memory=False)
servicios.fecha_captura = pd.to_datetime(servicios.fecha_captura, format='%d/%m/%Y', errors='ignore')

In [29]:
victimas = pd.read_csv('reportes/reporte_semujeres_10_febrero_2022.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [30]:
victimas.dropna(subset=['Dependencia de recepcion'],inplace=True)

In [31]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)
victimas['Feminicida']=0
fems = victimas[victimas['descripcion_otro_tipos']=='FEMINICIDA'].index
victimas.loc[fems,'Feminicida'] = 1
#victimas.loc[fems,'Otro'] = 0
victimas.drop(columns='Otro', inplace=True)

In [32]:
victimas = pd.concat([victimas, victimas.ACTIVIDAD.str.get_dummies(sep=',')], axis=1)

In [33]:
validez = victimas[victimas.escolaridad=='Estudios que no requieren validéz oficial'].index
tecnica = victimas[victimas.escolaridad=='Carrera técnica comercial'].index
victimas.loc[validez,'escolaridad']='Sin validez'
victimas.loc[tecnica,'escolaridad']='Carrera técnica'

In [34]:
victimas['Dependencia de recepcion'] = victimas['Dependencia de recepcion'].str.slice(start=6)
juzgados = victimas[victimas['Dependencia de recepcion'].str.contains('TSJ-JUZGADO')==True].index
victimas.loc[juzgados,'Dependencia de recepcion']='PODER JUDICIAL'

In [35]:
no = victimas[victimas['Victima de Trata']=='Se desconoce'].index
victimas.loc[no, 'Victima de Trata']= 'No'

In [36]:
feminicidios = feminicidios.merge(victimas[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
feminicidios = feminicidios.drop_duplicates(keep='last')
feminicidios = feminicidios.reset_index(drop=True)
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO','FEMINICIDIO'])].index
feminicidios.loc[fem_agrav,'Sentencia']='FEMINICIDIO'
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO EN GRADO DE TENTATIVA','FEMINICIDIO EN GRADO DE TENTATIVA'])].index
feminicidios.loc[fem_agrav,'Sentencia']='TENTATIVA DE FEMINICIDIO'

In [37]:
subtipo = pd.read_csv('../BD-sucias/subtipo_10_febrero_2022.csv', low_memory=False)
subtipo.fecha_recepcion = pd.to_datetime(subtipo.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
subtipo.fecha_hechos = pd.to_datetime(subtipo.fecha_hechos, format='%d/%m/%Y', errors='ignore')
subtipo = subtipo.dropna()
selecciones = subtipo[subtipo.SubtipoOrd=='Seleccione'].index
subtipo.loc[selecciones, 'SubtipoOrd'] = 'No especificado'

In [38]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [39]:
# censo poblacion y vivienda 2020 inegi
inegi_2020 = pd.read_csv('censo_yuc.csv')
pob_fem = inegi_2020[inegi_2020.NOM_LOC=='Total del Municipio'][['MUN','NOM_MUN','POBFEM' ]].reset_index(drop=True)

In [40]:
discapacidad = pd.read_csv('../BD-sucias/discapacidad.csv',low_memory=False)
discapacidad.fecha_recepcion = pd.to_datetime(discapacidad.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
discapacidad = discapacidad.iloc[:, [0,1,3,22,60,61,62,63,64,65,66,67]]
discapacidad['Dependencia de recepcion'] = discapacidad['Dependencia de recepcion'].str.slice(start=6)
discapacidad = discapacidad.rename(columns={'\ncaminar, subir o bajar sus pies?': 'Motriz (piernas)', 'ver (aunque use lentes)?': 'Visual','mover o usar brazos o manos?*':'Motriz (brazos)',
'aprender, recordar o concentrarse?':'Intelectual','escuchar (aunque use aparato auditivo)?':'Auditiva','bañarse, vestirse o comer?':'Psicosocial (independencia)',
'hablar o comunicarse (por ejemplo, entender, o ser entendido por otros)?':'Psicosocial (comunicación)',
'or problemas emocionales o mentales, ¿Cuánta dificultad tiene la víctima para realizar sus actividades diarias (con autonomía e independencia)? Problemas como: autismo, depresión, bipolaridad, esquizofrenia,':'Psicosocial (mentales)'})
discapacidad.drop_duplicates(['fk_euv'],keep='last', inplace=True)
discapacidad.reset_index(drop=True, inplace=True)
discapacidad.replace(to_replace='No indicado ', value='No tiene dificultad', inplace=True)

In [41]:
nan = victimas[victimas['Habla Indigena'].isna()==True].index
victimas.loc[nan, 'Habla Indigena'] = 'No'
des = victimas[victimas['Habla Indigena']=='Desconocido'].index
victimas.loc[des, 'Habla Indigena'] = 'No'

In [42]:
fiscalia = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("SCALÍA")==True].index
mujer = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("MUJER")==True].index
judicial = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("JUDICIAL")==True].index
salud = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("SALUD")==True].index
victimass = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("VÍCTIMA")==True].index
seguridad = discapacidad[discapacidad['Dependencia de recepcion'].str.contains("SEGURIDAD")==True].index
juzgados = discapacidad[discapacidad['Dependencia de recepcion'].str.contains('TSJ-JUZGADO')==True].index
discapacidad.loc[fiscalia, 'Dependencia de recepcion'] = 'FISCALÍA GENERAL DEL ESTADO'
discapacidad.loc[mujer, 'Dependencia de recepcion'] = 'SECRETARÍA DE LAS MUJERES'
discapacidad.loc[judicial, 'Dependencia de recepcion'] = 'PODER JUDICIAL'
discapacidad.loc[salud, 'Dependencia de recepcion'] = 'SECRETARÍA DE SALUD'
discapacidad.loc[victimass, 'Dependencia de recepcion'] = ' COMISIÓN EJECUTIVA ESTATAL DE ATENCIÓN A VÍCTIMAS'
discapacidad.loc[seguridad, 'Dependencia de recepcion'] = 'SECRETARÍA DE SEGURIDAD PÚBLICA'
discapacidad.loc[juzgados,'Dependencia de recepcion']='PODER JUDICIAL'

# Dashboard

## Layout

In [43]:
app.layout = html.Div(
    children=[
        html.H1(children="SEMUJERES TABLERO",),
        html.P(
            children="Texto de prueba",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Dependencia de recepcion", className="menu-title"),
                        dcc.Dropdown(
                            id="dependencia-filter",
                            options=[
                                {"label": dep, "value": dep}
                                for dep in (victimas['Dependencia de recepcion'].unique())
                            ],
                            value='SECRETARÍA DE LAS MUJERES',
                            multi=False,
                            clearable=False,
                            #className="dropdown",
                        ),
                    ]
                ),
                #html.Div(
                    #children=[
                        #html.Div(children="Municipio", className="menu-title"),
                        #dcc.Dropdown(
                            #id="municipio-filter",
                            #options=[
                                #{"label": mun, "value": mun}
                                #for mun in victimas.municipiohechos.unique()
                            #],
                            #value=[mun for mun in victimas.municipiohechos.unique()],
                            #multi=True,
                            #clearable=True,
                            #searchable=True,
                            #className="dropdown",
                        #),
                    #],
                #),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=victimas.fecha_recepcion.min(),
                            max_date_allowed=victimas.fecha_recepcion.max(),
                            start_date=victimas.fecha_recepcion.min(),
                            end_date=victimas.fecha_recepcion.max(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        dcc.Graph(
            id = 'map_1',  
        ),
        dcc.Graph(
            id = 'map_2',  
        ),
        dcc.Graph(
            id = 'fig_1',  
        ),
        dcc.Graph(
            id = 'fig_2',  
        ),
        dcc.Graph(
            id = 'fig_3',  
        ),
        dcc.Graph(
            id = 'fig_8',  
        ),
        dcc.Graph(
            id = 'fig_4',  
        ),
        dcc.Graph(
            id = 'fig_5',  
        ),
        dcc.Graph(
            id = 'fig_6',  
        ),
        dcc.Graph(
            id = 'fig_7',  
        ),
        dcc.Graph(
            id = 'fig_9',  
        ),
        dcc.Graph(
            id = 'fig_10',  
        ),
        dcc.Graph(
            id = 'fig_11',  
        ),
        dcc.Graph(
            id = 'fig_12',  
        ),
        dcc.Graph(
            id = 'fig_13',  
        ),
        dcc.Graph(
            id = 'fig_14',  
        ),
        dcc.Graph(
            id = 'fig_15',  
        ),
    ]
)

In [44]:
@app.callback(
    [   Output('fig_1', "figure"),
        Output('map_1', "figure"),
        Output('fig_2', "figure"),
        Output('map_2', "figure"),
        Output('fig_3', "figure"),
        Output('fig_4', "figure"),
        Output('fig_5', "figure"),
        Output('fig_6', "figure"),
        Output('fig_7', "figure"),
        Output('fig_8', "figure"),
        Output('fig_9', "figure"),
        Output('fig_10', "figure"),
        Output('fig_11', "figure"),
        Output('fig_12', "figure"),
        Output('fig_13', "figure"),
        Output('fig_14', "figure"),
        Output('fig_15', "figure"),
    ],
    [
        Input("dependencia-filter", "value"),
        #Input("municipio-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ]
)
def update_charts(dependencia, start_date, end_date):
    mask = (
        (victimas['Dependencia de recepcion'] == dependencia)
        #& (victimas['municipiohechos'].isin(municipio) ==True)
        & (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )

    filtered_data = victimas.loc[mask, :]
    
    # Fig 1 - tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(72,78)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    y = tipos[0].values
    y_total = len(filtered_data)

    fig_1 = px.bar(x=tipos.index,
                y=tipos[0],
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:6],
                color_discrete_map="identity",
                title='Gráfica 1. Casos de violencia hacia las mujeres por tipo de violencia'
                )
    fig_1.update_xaxes(type='category')
    fig_1.update_layout( xaxis_title=None)
    fig_1.update_traces(texttemplate='%{text} %')
    fig_1.update_traces(hovertemplate='Tipo de violencia=%{x}<br>Número de casos=%{y}<br>Porcentaje=%{text} %<extra></extra>')

    # Map 1 - casos x mun
    gdf = geopandas.GeoDataFrame(yuc, geometry='geometry_mun')
    
    mun_counts = pd.DataFrame(filtered_data.municipiohechos.value_counts())
    mun_counts.reset_index(inplace=True)
    
    gdf = pd.DataFrame(gdf.merge(mun_counts,left_on='nom_mun',right_on='index',how='left'))
    gdf = geopandas.GeoDataFrame(gdf, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.rename(columns={'municipiohechos':'Casos registrados'})
    gdf['Casos registrados log'] = np.log10(gdf['Casos registrados'] )
    gdf_crs = gdf.to_crs(pyproj.CRS.from_epsg(4326))
    #gdf_crs = gdf_crs.fillna(0)

    map_1 = px.choropleth(gdf_crs, geojson=gdf_crs.geometry_mun, 
                    locations=gdf_crs.index, color="Casos registrados log",
                    height=500,
                   color_continuous_scale="Viridis",
                    labels = {'nom_mun':'Municipio', "Casos registrados log":'log10'},
                    hover_data={"Casos registrados":True, "Casos registrados log":False},
                    title='Mapa 1. Casos Registrados por Municipio',
                   )
    map_1.update_geos(fitbounds="locations", visible=False)

    map_1.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

    map_1.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(filtered_data.año_recepcion),max(filtered_data.año_recepcion),len(filtered_data)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_1.update(layout = dict(title=dict(x=0.5)))
    map_1.update_layout(
        margin={"r":0,"t":30,"l":10,"b":10},
        coloraxis_colorbar={
            'title':'Escala'})
    map_1.update_layout(title_y=1, title_x=0)
    map_1.update_layout(coloraxis_showscale=True)

    # Map 2 - Tasa 1000
    dep = victimas[victimas['Dependencia de recepcion']==dependencia]

    mun_counts = pd.DataFrame(dep.municipiohechos.value_counts())
    mun_counts.reset_index(inplace=True)

    dep_2020 = victimas[victimas['Dependencia de recepcion']==dependencia]
    dep_2020 = dep_2020[dep_2020.año_recepcion==2020]

    mun_counts_2020 = pd.DataFrame(dep_2020.municipiohechos.value_counts())
    mun_counts_2020.reset_index(inplace=True)

    gdf = geopandas.GeoDataFrame(yuc, geometry='geometry_mun')
    gdf_2020 = pd.DataFrame(gdf.merge(mun_counts_2020,left_on='nom_mun',right_on='index',how='left'))
    gdf_2020 = pd.DataFrame(gdf_2020.merge(pob_fem,left_on='nom_mun',right_on='NOM_MUN',how='left'))
    gdf_2020 = geopandas.GeoDataFrame(gdf_2020, geometry='geometry_mun')
    gdf_2020 = gdf_2020.set_index('nom_mun')
    gdf_2020 = gdf_2020.rename(columns={'municipiohechos':'Casos registrados'})
    gdf_2020['Tasa 1000 casos'] = round(gdf_2020['Casos registrados']/gdf_2020['POBFEM'].astype(int)*1000,2)
    gdf_crs_2020 = gdf_2020.to_crs(pyproj.CRS.from_epsg(4326))
    gdf_crs_2020['Tasa 1000 casos'] = gdf_crs_2020['Tasa 1000 casos'].fillna(0)
    gdf_crs_2020['Casos registrados'] = gdf_crs_2020['Casos registrados'].fillna(0)

    map_2 = px.choropleth(gdf_crs_2020, geojson=gdf_crs_2020.geometry_mun, 
                    locations=gdf_crs_2020.index, color="Tasa 1000 casos",
                    #height=500,
                    #color_continuous_scale="Viridis",
                    #color_continuous_midpoint=np.average(gdf_crs_2020['Tasa 1000 casos'], weights=gdf_crs_2020['POBFEM'].astype(int)),
                    labels = {'nom_mun':'Municipio', 'POBFEM':'Población Femenina del Municipio', 'Tasa 1000 casos':'Tasa'},
                    hover_data={"Casos registrados":True,"POBFEM":True,},
                    title='Mapa 2. Tasa de casos registrados por cada 1000 mujeres por Municipio en 2020',
                    )
    map_2.update_geos(fitbounds="locations", visible=False)

    map_2.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

    map_2.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(2020,len(dep_2020)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_2.update(layout = dict(title=dict(x=0.5)))
    map_2.update_layout(
        margin={"r":0,"t":30,"l":10,"b":10},
        coloraxis_colorbar={
            'title':'Tasa'})
    map_2.update_layout(title_y=1, title_x=0)
    map_2.update_layout(coloraxis_showscale=True)

    # fig 2 - estudios
    y_2 = filtered_data.escolaridad.value_counts().values
    y_total_2 = sum(y_2)
    colors = len(y_2)
    fig_2 = px.bar(x=filtered_data.escolaridad.value_counts().index,
                    y=filtered_data.escolaridad.value_counts().values,
                    text= np.round(y_2/y_total_2*100,2),
                    labels = {'x': 'Escolaridad', "y":'Número de casos', 'text':'Porcentaje'},
                    color=px.colors.qualitative.Prism[:colors],
                    color_discrete_map="identity",
                    title='Gráfica 2. Estudios Concluidos'
                    )
    fig_2.update_xaxes(type='category')
    fig_2.update_layout(xaxis_title=None)
    fig_2.update_traces(texttemplate='%{text} %')

    # Fig 3 - Civil
    y = filtered_data.EstadoCivil.value_counts().values
    y_total = sum(y)
    colors = len(y)
    fig_3 = px.bar(x=filtered_data.EstadoCivil.value_counts().index,
                y=filtered_data.EstadoCivil.value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Estado Civil', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 3. Estado Civil'
                )
    fig_3.update_xaxes(type='category')
    fig_3.update_layout(xaxis_title=None)
    fig_3.update_traces(texttemplate='%{text} %')

    # Fig 4 - Discapacidad
    if dependencia in discapacidad['Dependencia de recepcion'].unique():
        discapacidades_tabla = pd.DataFrame()
        mask_dis = (
        (discapacidad['Dependencia de recepcion'] == dependencia)
        & (discapacidad.fecha_recepcion >= start_date)
        & (discapacidad.fecha_recepcion <= end_date)
        )
        discapacidad_dep = discapacidad.loc[mask_dis, :]
        discapacidad_tipos = discapacidad_dep.iloc[:, 4:]
        if len(discapacidad_tipos)!=0:
            for col in discapacidad_tipos.columns:
                new_df = pd.DataFrame(discapacidad_tipos[col].value_counts()).unstack().reset_index()
                discapacidades_tabla=discapacidades_tabla.append(new_df, ignore_index=True)
            discapacidades_tabla = discapacidades_tabla.rename(columns={'level_0':'Tipo de Discapacidad','level_1':'Nivel de dificultad',0:'Cuenta'})
            discapacidades_tabla.replace(to_replace='Motriz (brazos)', value='Motriz', inplace=True)
            discapacidades_tabla.replace(to_replace='Motriz (piernas)', value='Motriz', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (comunicación)', value='Psicosocial', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (independencia)', value='Psicosocial', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (mentales)', value='Psicosocial', inplace=True)
            discapacidades_tabla['Porcentaje']=round(discapacidades_tabla.Cuenta/len(discapacidad_tipos)*100,2)
            discapacidades_tabla = discapacidades_tabla[discapacidades_tabla['Nivel de dificultad']!='No tiene dificultad']
            total_mujeres = pd.DataFrame(discapacidades_tabla.groupby('Tipo de Discapacidad').Cuenta.sum()).sort_values('Cuenta', ascending=False)
            discapacidades_tabla=discapacidades_tabla.groupby(['Tipo de Discapacidad','Nivel de dificultad']).sum().reset_index()
            discapacidades_tabla['Tipo de Discapacidad'] = pd.Categorical(discapacidades_tabla['Tipo de Discapacidad'], total_mujeres.index)
            discapacidades_tabla= discapacidades_tabla.sort_values('Tipo de Discapacidad').reset_index(drop=True)
            
            fig_4 = px.bar(discapacidades_tabla, x="Tipo de Discapacidad", y="Porcentaje", color="Nivel de dificultad", 
            title="Gráfica 5. Número de mujeres con discapacidad",
            hover_name='Tipo de Discapacidad',
            text= "Cuenta",
            hover_data={"Cuenta":True, 'Tipo de Discapacidad':False},
            labels = {'Cuenta': 'Número de mujeres'},
            color_discrete_sequence=px.colors.qualitative.Prism[:5])
            #fig_4.update_layout(height=500)
            fig_4.update_traces(textposition='outside', cliponaxis=False)
            fig_4.update_layout(yaxis_ticksuffix = '%')
        else:
            fig_4=px.bar(title="Gráfica 5. Número de mujeres con discapacidad: SIN INFORMACION EN ESTAS FECHAS") 
    else:
        fig_4=px.bar(title="Gráfica 5. Número de mujeres con discapacidad: SIN INFORMACION PARA ESTA DEPENDENCIA")
        
    # Fig 5 - Indigena
    y = filtered_data['Habla Indigena'].value_counts().values
    y_total = sum(y)
    prismas = px.colors.qualitative.Prism 
    prismas += px.colors.qualitative.Prism
    colors=len(y)
    fig_5 = px.bar(x=filtered_data['Habla Indigena'].value_counts().index,
                y=filtered_data['Habla Indigena'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Habla lengua indígena', "y":'Número de casos', 'text':'Porcentaje'},
                color=prismas[:colors],
                color_discrete_map="identity",
                title='Gráfica 6. Mujeres que hablan Lengua Indígena'
                )
    fig_5.update_xaxes(type='category')
    fig_5.update_layout(xaxis_title=None, height=500)
    fig_5.update_traces(texttemplate='%{text} %')

    # Fig 6 - Conocimiento autoridad
    y = filtered_data['Conocimiento de autoridad'].value_counts().values
    y_total = sum(y)
    colors = len(y)
    fig_6 = px.bar(x=filtered_data['Conocimiento de autoridad'].value_counts().index,
                y=filtered_data['Conocimiento de autoridad'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Conocimiento de alguna autoridad', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 7. Casos que son conocidos por alguna autoridad'
                )
    fig_6.update_xaxes(type='category')
    fig_6.update_layout( xaxis_title=None)
    fig_6.update_traces(texttemplate='%{text} %')

    # Fig 7 - Autoridad conoce caso
    si_conocimiento = filtered_data[filtered_data['Conocimiento de autoridad']=='Si']
    mp = si_conocimiento['Descripcion de autoridad'].str.contains("MINISTERIO").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("MP").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("M.P").sum() 
    fg = si_conocimiento['Descripcion de autoridad'].str.contains("FIS").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("FG").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("F.G").sum()
    juz = si_conocimiento['Descripcion de autoridad'].str.contains("JUZ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JUEZ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JZ").sum()
    agencia = si_conocimiento['Descripcion de autoridad'].str.contains("AGE").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("AG").sum()
    cjm = si_conocimiento['Descripcion de autoridad'].str.contains("JUS").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("CJ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JM").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("B1").sum()
    prod = si_conocimiento['Descripcion de autoridad'].str.contains("PRODE").sum()
    salud = si_conocimiento['Descripcion de autoridad'].str.contains("SALUD").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("HOSP").sum()
    judicial = si_conocimiento['Descripcion de autoridad'].str.contains("MUN").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("SSP").sum()
    judicial_juzgado = judicial+juz
    fis_agencia = fg + agencia
    dic_autoridades = { 'Autoridad':['Ministerio Público','Fiscalía','PRODENNAY', 'CJM','Centro de Salud', 'Poder Judicial'],
                    #'Porcentaje':100*np.array([mp, fis_agencia,  prod,  cjm, salud,judicial_juzgado])/sum([mp, fis_agencia,  prod,  cjm, salud, judicial_juzgado]),
                    'Porcentaje':100*np.array([mp, fis_agencia,  prod,  cjm, salud,judicial_juzgado])/len(si_conocimiento),
                    'Count': np.array([mp, fis_agencia, prod,  cjm, salud, judicial_juzgado])}
    autoridades = pd.DataFrame(dic_autoridades)
    autoridades = autoridades.sort_values('Count', ascending=False)
    y = autoridades['Count']
    y_total = len(si_conocimiento)
    fig_7 = px.bar(x=autoridades['Autoridad'],
                y=autoridades['Count'],
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Autoridad', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:6],
                color_discrete_map="identity",
                title='Gráfica 8. Autoridad que conoce el caso'
                )
    fig_7.update_xaxes(type='category')
    fig_7.update_layout( xaxis_title=None,)
    fig_7.update_traces(texttemplate='%{text} %')

    # Fig 8 - Actividad que realizan las mujeres
    y = [filtered_data['Trabaja en el hogar'].sum(), filtered_data['Trabaja fuera del hogar'].sum(), filtered_data['Estudia'].sum(), filtered_data['Jubilada/Pensionada'].sum()+filtered_data['Pensionada'].sum(),filtered_data['Otro'].sum(),filtered_data['Se desconoce'].sum()]
    #y_total = sum(y)
    y_total = np.array(len(filtered_data), dtype=int)
    fig_8 = px.bar(x=['Trabaja en el hogar', 'Trabaja fuera del hogar','Estudia','Jubilada/Pensionada','Otro','Se desconoce'], 
                y = y,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Actividad', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:6],
                color_discrete_map="identity",
                title='Gráfica 4. Actividad que realizan las mujeres'
                )
    fig_8.update_xaxes(type='category')
    fig_8.update_layout( xaxis={'categoryorder':'total descending'}, xaxis_title=None,)
    #fig.update_traces(texttemplate='%{text} %', textposition='outside')
    fig_8.update_traces(texttemplate='%{text} %')

    # Fig 9 - Modalidad
    y = filtered_data.modalidad.value_counts().values
    y_total = sum(y)
    colors = len(y)
    fig_9 = px.bar(x=filtered_data.modalidad.value_counts().index,
                y=filtered_data.modalidad.value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Modalidad de la violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 9. Modalidad de la violencia'
                )
    fig_9.update_xaxes(type='category')
    fig_9.update_layout( xaxis_title=None,)
    fig_9.update_traces(texttemplate='%{text} %')

    # Fig 10 - vinculo victima
    y = filtered_data['Tipo de vínculo con victima'].value_counts().values
    y_total = sum(y)
    colors = len(y)
    fig_10 = px.bar(x=filtered_data['Tipo de vínculo con victima'].value_counts().index,
                y=filtered_data['Tipo de vínculo con victima'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Tipo de vínculo con la víctima', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 10. Tipo de vínculo con la víctima'
                )
    fig_10.update_xaxes(type='category')
    fig_10.update_layout( xaxis_title=None)
    fig_10.update_traces(texttemplate='%{text} %')

    # Fig 11 - detalle vinculo victima
    y = filtered_data['Detalle del Tipo Vínculo con victima'].value_counts().values
    y_total = sum(y) 
    colors = len(y)
    fig_11 = px.bar(x=filtered_data['Detalle del Tipo Vínculo con victima'].value_counts().index,
                y=filtered_data['Detalle del Tipo Vínculo con victima'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Detalle del vínculo con la víctima', "y":'Número de casos', 'text':'Porcentaje'},
                color=prismas[:colors],
                color_discrete_map="identity",
                title='Gráfica 11. Detalle del vínculo con la víctima'
                )
    fig_11.update_xaxes(type='category')
    fig_11.update_layout( xaxis={'categoryorder':'total descending'})
    fig_11.update_layout( xaxis_title=None)
    fig_11.update_traces(texttemplate='%{text} %')

    # Fig 12 - Violencia ultimo año
    tuvo_violencia = filtered_data[(filtered_data['Detalle del Tipo Vínculo con victima'].isin(['Cónyuge o pareja ', 'Ex pareja']))&(filtered_data['fecha_recepcion']>=(filtered_data.fecha_recepcion.max()-pd.offsets.DateOffset(years=1)))]
    ultimo_año = filtered_data[filtered_data['fecha_recepcion']>=(filtered_data.fecha_recepcion.max()-pd.offsets.DateOffset(years=1))]
    ultimo_año['violencia_ex_pareja'] = 'No'
    ultimo_año.loc[tuvo_violencia.index, 'violencia_ex_pareja'] = 'Si'
    y = ultimo_año['violencia_ex_pareja'].value_counts().values
    y_total = len(ultimo_año)
    colors = len(y)
    fig_12 = px.bar(x=ultimo_año['violencia_ex_pareja'].value_counts().index,
                y=ultimo_año['violencia_ex_pareja'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Victima de violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title="""Gráfica 12. Mujeres que han experimentado violencia por parte de su última pareja en los últimos 12 meses<br><b>Total de casos en los últimos 12 meses:</b> {}""".format(len(ultimo_año))
                )
    fig_12.update_xaxes(type='category')
    fig_12.update_layout( xaxis_title=None,)
    fig_12.update_traces(texttemplate='%{text} %')

    # Fig 13 - Victima de trata
    y = filtered_data['Victima de Trata'].value_counts().values
    y_total = sum(y)
    colors = len(y)
    fig_13 = px.bar(x= filtered_data['Victima de Trata'].value_counts().index,
                y= filtered_data['Victima de Trata'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Victima de Trata', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 13. Casos relacionados con el delito de Trata'
                )
    fig_13.update_xaxes(type='category')
    fig_13.update_layout( xaxis_title=None)
    fig_13.update_traces(texttemplate='%{text} %')

    # Fig 14 - Delincuencia\
    y = filtered_data['Victima de delincuencia'].value_counts().values
    y_total = sum(y)
    colors=len(y)
    fig_14 = px.bar(x=filtered_data['Victima de delincuencia'].value_counts().index,
                y=filtered_data['Victima de delincuencia'].value_counts().values,
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Victima de delincuencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:colors],
                color_discrete_map="identity",
                title='Gráfica 14. Casos relacionados con Delincuencia Organizada'
                )
    fig_14.update_xaxes(type='category')
    fig_14.update_layout( xaxis_title=None)
    fig_14.update_traces(texttemplate='%{text} %')
    #fig.update_xaxes(visible=False, showticklabels=False)
    fig_14.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [0, 1],
            ticktext = ['No','Si']
        )
    )

    # Fig 15 - Servicios brindados
    if dependencia in servicios['dependenciaquebrindoservicio'].unique():
        mask_ser = (
        (servicios['dependenciaquebrindoservicio'] == dependencia)
        & (servicios.fecha_captura >= start_date)
        & (servicios.fecha_captura <= end_date)
        )
        servicios_data = servicios.loc[mask_ser, :]
        if len(servicios_data)!=0:
            y = servicios_data.serviciodetalle.value_counts().values
            y_total = sum(y)
            colors = len(y)
            fig_15 = px.bar(x=servicios_data.serviciodetalle.value_counts().index,
                        y=servicios_data.serviciodetalle.value_counts().values,
                        text= np.round(y/y_total*100,2),
                        labels = {'x': 'Servicio proporcionado', "y":'Número de casos', 'text':'Porcentaje'},
                        color=prismas[:colors],
                        color_discrete_map="identity",
                        title='Gráfica 15. Servicios proporcionados por la dependencia'
                        )
            fig_15.update_xaxes(type='category')
            fig_15.update_layout( xaxis={'categoryorder':'total descending'},)
            fig_15.update_layout( xaxis_title=None)
            fig_15.update_traces(texttemplate='%{text} %')
        else:
            fig_15=px.bar(title='Gráfica 15. Servicios proporcionados por la dependencia: SIN INFORMACION PARA ESTAS FECHAS')
    else:
        fig_15=px.bar(title='Gráfica 15. Servicios proporcionados por la dependencia: SIN INFORMACION PARA ESTA DEPENDENCIA')
    
    return fig_1,map_1,fig_2,map_2,fig_3,fig_4,fig_5, fig_6, fig_7, fig_8,fig_9,fig_10,fig_11,fig_12, fig_13, fig_14, fig_15

In [45]:
"""mask = (
        (victimas['Dependencia de recepcion'] == 'SECRETARÍA DE LAS MUJERES')
        #& (victimas['municipiohechos'].isin(municipio) ==True)
        & (victimas.fecha_recepcion >= '07-08-2021')
        & (victimas.fecha_recepcion <= victimas.fecha_recepcion.max())
    )
filtered_data = victimas.loc[mask, :]"""

"""dependencia = 'SECRETARÍA DE LAS MUJERES'

if dependencia in discapacidad['Dependencia de recepcion'].unique():

        discapacidades_tabla = pd.DataFrame()
        mask_dis = (
        (discapacidad['Dependencia de recepcion'] == dependencia)
        & (discapacidad.fecha_recepcion >= '09-01-2021')
        & (discapacidad.fecha_recepcion <= victimas.fecha_recepcion.max())
        )
        discapacidad_dep = discapacidad.loc[mask_dis, :]
        discapacidad_tipos = discapacidad_dep.iloc[:, 4:]
        if len(discapacidad_tipos)!=0:
            for col in discapacidad_tipos.columns:
                new_df = pd.DataFrame(discapacidad_tipos[col].value_counts()).unstack().reset_index()
                discapacidades_tabla=discapacidades_tabla.append(new_df, ignore_index=True)
            discapacidades_tabla = discapacidades_tabla.rename(columns={'level_0':'Tipo de Discapacidad','level_1':'Nivel de dificultad',0:'Cuenta'})
            discapacidades_tabla.replace(to_replace='Motriz (brazos)', value='Motriz', inplace=True)
            discapacidades_tabla.replace(to_replace='Motriz (piernas)', value='Motriz', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (comunicación)', value='Psicosocial', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (independencia)', value='Psicosocial', inplace=True)
            discapacidades_tabla.replace(to_replace='Psicosocial (mentales)', value='Psicosocial', inplace=True)
            discapacidades_tabla['Porcentaje']=round(discapacidades_tabla.Cuenta/len(discapacidad_tipos)*100,2)
            discapacidades_tabla = discapacidades_tabla[discapacidades_tabla['Nivel de dificultad']!='No tiene dificultad']
            total_mujeres = pd.DataFrame(discapacidades_tabla.groupby('Tipo de Discapacidad').Cuenta.sum()).sort_values('Cuenta', ascending=False)
            discapacidades_tabla=discapacidades_tabla.groupby(['Tipo de Discapacidad','Nivel de dificultad']).sum().reset_index()
            discapacidades_tabla['Tipo de Discapacidad'] = pd.Categorical(discapacidades_tabla['Tipo de Discapacidad'], total_mujeres.index)
            discapacidades_tabla= discapacidades_tabla.sort_values('Tipo de Discapacidad').reset_index(drop=True)
            
            fig_4 = px.bar(discapacidades_tabla, x="Tipo de Discapacidad", y="Porcentaje", color="Nivel de dificultad", 
            title="Gráfica 5. Número de mujeres con discapacidad",
            hover_name='Tipo de Discapacidad',
            text= "Cuenta",
            hover_data={"Cuenta":True, 'Tipo de Discapacidad':False},
            labels = {'Cuenta': 'Número de mujeres'},
            color_discrete_sequence=px.colors.qualitative.Prism[:5])
            #fig_4.update_layout(height=500)
            fig_4.update_traces(textposition='outside', cliponaxis=False)
            fig_4.update_layout(yaxis_ticksuffix = '%')
        else:
            fig_4=px.bar(title="Gráfica 5. Número de mujeres con discapacidad: SIN INFORMACION PARA ESTAS FECHAS")
else:
        fig_4=px.bar(title="Gráfica 5. Número de mujeres con discapacidad: SIN INFORMACION")
fig_4.show()"""

'dependencia = \'SECRETARÍA DE LAS MUJERES\'\n\nif dependencia in discapacidad[\'Dependencia de recepcion\'].unique():\n\n        discapacidades_tabla = pd.DataFrame()\n        mask_dis = (\n        (discapacidad[\'Dependencia de recepcion\'] == dependencia)\n        & (discapacidad.fecha_recepcion >= \'09-01-2021\')\n        & (discapacidad.fecha_recepcion <= victimas.fecha_recepcion.max())\n        )\n        discapacidad_dep = discapacidad.loc[mask_dis, :]\n        discapacidad_tipos = discapacidad_dep.iloc[:, 4:]\n        if len(discapacidad_tipos)!=0:\n            for col in discapacidad_tipos.columns:\n                new_df = pd.DataFrame(discapacidad_tipos[col].value_counts()).unstack().reset_index()\n                discapacidades_tabla=discapacidades_tabla.append(new_df, ignore_index=True)\n            discapacidades_tabla = discapacidades_tabla.rename(columns={\'level_0\':\'Tipo de Discapacidad\',\'level_1\':\'Nivel de dificultad\',0:\'Cuenta\'})\n            discapacidades_

In [46]:
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\carol\AppData\Local\Temp/ipykernel_20084/3512330574.py:370: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\carol\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

